# Logger Wrapper personalization/configuration

We will see on this notebook how to personalize the logger wrapper defined by sinergym.

In [1]:
import gymnasium as gym
import numpy as np
import sinergym
from sinergym.utils.wrappers import (LoggerWrapper, MultiObsWrapper,
                                     NormalizeObservation)
from sinergym.utils.constants import RANGES_5ZONE

## Step 1 Inherit and modify the CSVloger

First we need to change the CSV logger to modify the values written into the file on the function create_row_contents

In [2]:

from sinergym.utils.logger import CSVLogger
from typing import Any, Dict, Optional, Sequence, Tuple, Union, List

class CustomCSVLogger(CSVLogger):

    def __init__(
            self,
            monitor_header: str,
            progress_header: str,
            log_progress_file: str,
            log_file: Optional[str] = None,
            flag: bool = True):
        super(CustomCSVLogger, self).__init__(monitor_header,progress_header,log_progress_file,log_file,flag)
        self.last_10_steps_reward = [0]*10

    def _create_row_content(
            self,
            obs: List[Any],
            action: Union[int, np.ndarray, List[Any]],
            reward: Optional[float],
            done: bool,
            info: Optional[Dict[str, Any]]) -> List:
            
        if reward is not None:
            self.last_10_steps_reward.pop(0)
            self.last_10_steps_reward.append(reward)

        if info is None:  # In a reset
            return [0] + list(obs) + list(action) + \
                [0, reward, np.mean(self.last_10_steps_reward), None, None, None, done]
        else:
            return [
                info['timestep']] + list(obs) + list(action) + [
                info['time_elapsed'],
                reward,
                np.mean(self.last_10_steps_reward),
                info['total_power_no_units'],
                info['comfort_penalty'],
                info['abs_comfort'],
                done]

    

## Step 2 Instantiate the LoggerWrapper


now we need to instantiate the loggerwrapper and specify the new headers of our file and the csvlogger class we want to use.

In [3]:
env=gym.make('Eplus-demo-v1')
env=LoggerWrapper(env,logger_class=CustomCSVLogger,monitor_header = ['timestep'] + env.variables['observation'] +
                env.variables['action'] + ['time (seconds)', 'reward', '10-mean-reward',
                'power_penalty', 'comfort_penalty', 'done'])

[2023-02-09 11:23:44,589] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2023-02-09 11:23:44,590] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-02-09 11:23:44,591] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-02-09 11:23:44,592] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...
[2023-02-09 11:23:44,592] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Setting up action definition in building model if exists...


Now, you can see in Sinergym output folder that you will have available `progress.csv` file and `monitor.csv` files in each episode.

In [4]:
for i in range(1):
    obs, info = env.reset()
    rewards = []
    terminated = False
    current_month = 0
    while not terminated:
        a = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(a)
        rewards.append(reward)
        if info['month'] != current_month:  # display results every month
            current_month = info['month']
            print('Reward: ', sum(rewards), info)
    print('Episode ', i, 'Mean reward: ', np.mean(
        rewards), 'Cumulative reward: ', sum(rewards))
env.close()

[2023-02-09 11:23:49,091] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-02-09 11:23:49,099] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-demo-v1-res1/Eplus-env-sub_run1
Reward:  -0.5795042490937325 {'timestep': 1, 'time_elapsed': 900, 'year': 1991, 'month': 1, 'day': 1, 'hour': 0, 'action': [19, 26], 'total_power': 4954.294078583251, 'total_power_no_units': -0.49542940785832507, 'comfort_penalty': -0.6635790903291401, 'abs_comfort': 0.6635790903291401, 'temperatures': [19.33642090967086]}
Reward:  -2058.929866314376 {'timestep': 2976, 'time_elapsed': 2678400, 'year': 1991, 'month': 2, 'day': 1, 'hour': 0, 'action': [15, 30], 'total_power': 4284.524651309577, 'total_power_no_units': -0.42845246513095775, 'comfort_penalty': -1.7535396279158597, 'abs_comfort': 1.7535396279158597, 'temperatures': [18.24646037208414]}
Reward:  -4051.758361432438 {'timestep': 5664, 'time_elapsed'